In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [2]:
file_path = 'AirborneEmissions_Processed.xlsx'
excel_file = pd.ExcelFile(file_path)

In [3]:
future_years = np.arange(2023, 2031)  
output_file_path = 'predictions_ets.xlsx'  
images_output_dir = 'ets_images'  
os.makedirs(images_output_dir, exist_ok=True)

all_forecast_results = {}

In [4]:
for sheet_name in excel_file.sheet_names:
    sheet_df = pd.read_excel(file_path, sheet_name=sheet_name)
    print(f"Processing sheet: {sheet_name}, Number of Columns: {sheet_df.shape[1]}")
    
    years = sheet_df.columns[1:] 
    elements = sheet_df.iloc[:, 0]
    forecast_results = pd.DataFrame()

    for element in elements:
        historical_emissions = sheet_df.loc[sheet_df.iloc[:, 0] == element, years].values.flatten()
        
        try:
            model = ExponentialSmoothing(historical_emissions, trend='add', seasonal=None, initialization_method='estimated')
            model_fit = model.fit()
            
            forecast = model_fit.forecast(steps=len(future_years))
            
            total_emissions = np.concatenate([historical_emissions, forecast])
            forecast_results[element] = total_emissions
        
        except Exception as e:
            print(f"ETS model failed for element '{element}' in sheet '{sheet_name}': {e}")
    
    all_years = np.concatenate([years, future_years])
    forecast_results.index = all_years
    all_forecast_results[sheet_name] = forecast_results
    
    plt.figure(figsize=(10, 6))
    for element in elements:
        plt.plot(all_years, forecast_results[element], label=element)
    
    plt.xlabel("Year")
    plt.ylabel("Emissions")
    plt.legend(loc="upper left")
    plt.title(f"ETS Forecast of Future Emissions ({sheet_name})")
    
    image_file_path = os.path.join(images_output_dir, f"{sheet_name}_ets_forecast.png")
    plt.savefig(image_file_path)
    plt.close()  
    print(f"Graph saved for sheet '{sheet_name}' at {image_file_path}")

Processing sheet: Arsenic, Number of Columns: 34
Graph saved for sheet 'Arsenic' at ets_images\Arsenic_ets_forecast.png
Processing sheet: Cadmium, Number of Columns: 34


c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)
c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1412: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + k * 2
c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:1419: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + k * np.log(self.nobs)


Graph saved for sheet 'Cadmium' at ets_images\Cadmium_ets_forecast.png
Processing sheet: Chromium, Number of Columns: 34
Graph saved for sheet 'Chromium' at ets_images\Chromium_ets_forecast.png
Processing sheet: Copper, Number of Columns: 34
Graph saved for sheet 'Copper' at ets_images\Copper_ets_forecast.png
Processing sheet: Lead, Number of Columns: 34
Graph saved for sheet 'Lead' at ets_images\Lead_ets_forecast.png
Processing sheet: Mercury, Number of Columns: 34
Graph saved for sheet 'Mercury' at ets_images\Mercury_ets_forecast.png
Processing sheet: Nickel, Number of Columns: 34
Graph saved for sheet 'Nickel' at ets_images\Nickel_ets_forecast.png
Processing sheet: Selenium, Number of Columns: 34
Graph saved for sheet 'Selenium' at ets_images\Selenium_ets_forecast.png
Processing sheet: Vanadium, Number of Columns: 34
Graph saved for sheet 'Vanadium' at ets_images\Vanadium_ets_forecast.png
Processing sheet: Zinc, Number of Columns: 34
Graph saved for sheet 'Zinc' at ets_images\Zinc_e

In [5]:
with pd.ExcelWriter(output_file_path) as writer:
    for sheet_name, forecast_result in all_forecast_results.items():
        forecast_result.to_excel(writer, sheet_name=sheet_name)

print(f"ETS forecast results saved to {output_file_path}")

ETS forecast results saved to predictions_ets.xlsx
